<a href="https://colab.research.google.com/github/sallumandya1995/grammar_coorection/blob/main/course/en/chapter5/section6_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://www.youtube.com/watch?v=scJsty_DR3o
# https://www.numpyninja.com/post/question-answering-through-bert-in-10-steps
# https://huggingface.co/clips/mfaq
# https://www.youtube.com/watch?v=OlhNZg4gOvA

# https://haystack.deepset.ai/tutorials/01_basic_qa_pipeline

# https://colab.research.google.com/github/kstathou/vector_engine/blob/master/notebooks/001_vector_search.ipynb#scrollTo=aecWwyrutyef
# https://www.youtube.com/watch?v=w1dMEWm7jBc

In [ ]:
#to watch

# https://www.youtube.com/watch?v=ZxR38An5TQE


# https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb


In [ ]:
#dhiraj 

In [ ]:
import numpy as np 
import pandas as pd 
import spacy
import string
import gensim
import operator
import re


In [ ]:
df = pd.read_csv('/content/24.1_november_cleanedData.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3392 entries, 0 to 3391
Data columns (total 5 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Unnamed: 0                      3392 non-null   int64 
 1   title                           3392 non-null   object
 2   field_solutiondescription       3245 non-null   object
 3   field_solutiondescriptionimage  3373 non-null   object
 4   nid                             3392 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 132.6+ KB


In [ ]:
df2=df[['title','field_solutiondescription','field_solutiondescriptionimage','nid']]
df2.head()

,title,field_solutiondescription,field_solutiondescriptionimage,nid
0,$finance,sector financial servicesdata category financi...,/sites/default/files/img_35.jpg,6533
1,House_price_prediction_final,house_price_prediction_final,/sites/default/files/img_74.jpg,6532
2,Diabetes_Patients,sector lifescience healthcaredata category hea...,/sites/default/files/img_80.jpg,6531
3,21-Nov,sector business servicesdata category adaption...,/sites/default/files/img_85.jpg,6530
4,21-Nov,sector business servicesdata category adaption...,/sites/default/files/img_58.jpg,6529


In [ ]:
df2['field_solutiondescription'].head(6)

0    sector financial servicesdata category financi...
1                         house_price_prediction_final
2    sector lifescience healthcaredata category hea...
3    sector business servicesdata category adaption...
4    sector business servicesdata category adaption...
5                       heart failure prediction final
Name: field_solutiondescription, dtype: object

### Data Cleaning and Pre-processing



In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS

spacy_nlp = spacy.load('en_core_web_sm')

#create list of punctuations and stopwords
punctuations = string.punctuation
stop_words = spacy.lang.en.stop_words.STOP_WORDS

#function for data cleaning and processing
#This can be further enhanced by adding / removing reg-exps as desired.

def spacy_tokenizer(sentence):
 
    #remove distracting single quotes
    sentence = re.sub('\'','',str(sentence))

    #remove digits adnd words containing digits
    sentence = re.sub('\w*\d\w*','',sentence)

    #replace extra spaces with single space
    sentence = re.sub(' +',' ',sentence)

    #remove unwanted lines starting from special charcters
    sentence = re.sub(r'\n: \'\'.*','',sentence)
    sentence = re.sub(r'\n!.*','',sentence)
    sentence = re.sub(r'^:\'\'.*','',sentence)
    
    #remove non-breaking new line characters
    sentence = re.sub(r'\n',' ',sentence)
    
    #remove punctunations
    sentence = re.sub(r'[^\w\s]',' ',sentence)
    
    sentence = re.sub(r'_',' ',sentence)

    
    #creating token object
    tokens = spacy_nlp(sentence)
    
    #lower, strip and lemmatize
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    
    #remove stopwords, and exclude words less than 2 characters
    tokens = [word for word in tokens if word not in stop_words and word not in punctuations and len(word) > 2]
    
    #return tokens
    return tokens


In [ ]:
print ('Cleaning and Tokenizing...')
df2['field_solutiondescription_tokenized'] = df2['field_solutiondescription'].map(lambda x: spacy_tokenizer(x))
#df2['title_tokenized']=df2['title'].map(lambda x: spacy_tokenizer(x))

df2.head()

Cleaning and Tokenizing...


,title,field_solutiondescription,field_solutiondescriptionimage,nid,field_solutiondescription_tokenized
0,$finance,sector financial servicesdata category financi...,/sites/default/files/img_35.jpg,6533,"[sector, financial, servicesdata, category, fi..."
1,House_price_prediction_final,house_price_prediction_final,/sites/default/files/img_74.jpg,6532,"[house, price, prediction, final]"
2,Diabetes_Patients,sector lifescience healthcaredata category hea...,/sites/default/files/img_80.jpg,6531,"[sector, lifescience, healthcaredata, category..."
3,21-Nov,sector business servicesdata category adaption...,/sites/default/files/img_85.jpg,6530,"[sector, business, servicesdata, category, ada..."
4,21-Nov,sector business servicesdata category adaption...,/sites/default/files/img_58.jpg,6529,"[sector, business, servicesdata, category, ada..."


Store the tokenized column into a sepearte variable for ease of operations in subsequent sections and have a quick peek into the values

In [ ]:
#fsd=df2['title_tokenized']
fsd=df2['field_solutiondescription_tokenized']

#storing the tokenized col into separete variable for better operation


In [ ]:


fsd[0:5]


0    [sector, financial, servicesdata, category, fi...
1                    [house, price, prediction, final]
2    [sector, lifescience, healthcaredata, category...
3    [sector, business, servicesdata, category, ada...
4    [sector, business, servicesdata, category, ada...
Name: field_solutiondescription_tokenized, dtype: object

### Building Word Dictionary



In [ ]:
from gensim import corpora

#creating term dictionary
%time dictionary = corpora.Dictionary(fsd)

#filter out terms which occurs in less than 4 documents and more than 20% of the documents.
#NOTE: Since we have smaller dataset, we will keep this commented for now.

#dictionary.filter_extremes(no_below=4, no_above=0.2)

#list of few which which can be further removed
stoplist = set('hello and if this can would should could tell ask stop come go')
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
dictionary.filter_tokens(stop_ids)



CPU times: user 109 ms, sys: 1.99 ms, total: 111 ms
Wall time: 112 ms


In [ ]:
#print top 50 items from the dictionary with their unique token-id
dict_tokens = [[[dictionary[key], dictionary.token2id[dictionary[key]]] for key, value in dictionary.items() if key <= 50]]
print (dict_tokens)

[[['case', 0], ['category', 1], ['characteristic', 2], ['finance', 3], ['financial', 4], ['marketdata', 5], ['potential', 6], ['sector', 7], ['servicesdata', 8], ['undefine', 9], ['undefined', 10], ['final', 11], ['house', 12], ['prediction', 13], ['price', 14], ['care', 15], ['consist', 16], ['dataset', 17], ['diabetes', 18], ['health', 19], ['healthcaredata', 20], ['information', 21], ['life', 22], ['lifescience', 23], ['patient', 24], ['sciencedata', 25], ['vital', 26], ['actionsdata', 27], ['adaption', 28], ['business', 29], ['descr', 30], ['action', 31], ['data', 32], ['failure', 33], ['heart', 34], ['insight', 35], ['publish', 36], ['report', 37], ['consumer', 38], ['product', 39], ['retaildata', 40], ['test', 41], ['actionable', 42], ['advanced', 43], ['company', 44], ['current', 45], ['dashboard', 46], ['datum', 47], ['decision', 48], ['emerge', 49], ['empower', 50]]]


### Feature Extraction (Bag of Words)



In [ ]:
corpus = [dictionary.doc2bow(desc) for desc in fsd]

word_frequencies = [[(dictionary[id], frequency) for id, frequency in line] for line in corpus[0:3]]

print(word_frequencies)

[[('case', 1), ('category', 1), ('characteristic', 1), ('finance', 1), ('financial', 2), ('marketdata', 1), ('potential', 1), ('sector', 1), ('servicesdata', 1), ('undefine', 1), ('undefined', 1)], [('final', 1), ('house', 1), ('prediction', 1), ('price', 1)], [('case', 1), ('category', 1), ('characteristic', 1), ('potential', 1), ('sector', 1), ('undefine', 1), ('undefined', 1), ('care', 1), ('consist', 1), ('dataset', 1), ('diabetes', 1), ('health', 1), ('healthcaredata', 1), ('information', 1), ('life', 1), ('lifescience', 1), ('patient', 1), ('sciencedata', 1), ('vital', 1)]]


In [ ]:
print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)], [(11, 1), (12, 1), (13, 1), (14, 1)], [(0, 1), (1, 1), (2, 1), (6, 1), (7, 1), (9, 1), (10, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1)], [(0, 1), (1, 1), (2, 1), (6, 1), (7, 1), (8, 1), (9, 2), (27, 1), (28, 1), (29, 1), (30, 1)], [(0, 1), (1, 1), (2, 1), (6, 1), (7, 1), (8, 1), (9, 2), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1)], [(11, 1), (13, 1), (33, 1), (34, 1)], [(35, 1), (36, 1)], [(36, 1), (37, 1)], [(36, 1), (37, 1)], [(0, 1), (1, 1), (2, 1), (6, 1), (7, 1), (9, 1), (10, 1), (27, 1), (28, 1), (38, 1), (39, 1), (40, 1), (41, 1)], [(19, 1), (35, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 2), (65, 1), (66, 2), (67, 2), (68, 1), (69, 1)], [(70, 1)], [(46, 1), (5

The above results shows vocabulary with their frequency.

### Build Tf-Idf and LSI Model



In [ ]:
%time df2_tfidf_model = gensim.models.TfidfModel(corpus, id2word=dictionary)
%time df2_lsi_model = gensim.models.LsiModel(df2_tfidf_model[corpus], id2word=dictionary, num_topics=300)

CPU times: user 29.6 ms, sys: 987 µs, total: 30.5 ms
Wall time: 32.4 ms
CPU times: user 2.56 s, sys: 507 ms, total: 3.07 s
Wall time: 1.85 s


Serialize and Store the corpus locally for easy retrival whenver required.

In [ ]:
%time gensim.corpora.MmCorpus.serialize('df2_tfidf_model_mm', df2_tfidf_model[corpus])
%time gensim.corpora.MmCorpus.serialize('df2_lsi_model_mm',df2_lsi_model[df2_tfidf_model[corpus]])

CPU times: user 519 ms, sys: 63.5 ms, total: 582 ms
Wall time: 555 ms
CPU times: user 2.84 s, sys: 113 ms, total: 2.96 s
Wall time: 3 s


In [ ]:
#Load the indexed corpus
df2_tfidf_corpus = gensim.corpora.MmCorpus('df2_tfidf_model_mm')
df2_lsi_corpus = gensim.corpora.MmCorpus('df2_lsi_model_mm')

print(df2_tfidf_corpus)
print(df2_lsi_corpus)


MmCorpus(3392 documents, 5073 features, 68279 non-zero entries)
MmCorpus(3392 documents, 300 features, 939469 non-zero entries)


In [ ]:
#Semantic Search

In [ ]:
from gensim.similarities import MatrixSimilarity

%time df2_index = MatrixSimilarity(df2_lsi_corpus, num_features = df2_lsi_corpus.num_terms)

CPU times: user 1.07 s, sys: 15.8 ms, total: 1.09 s
Wall time: 1.09 s


Below is the helper function to search the index, sort and return the results

In [ ]:
from operator import itemgetter


def semantic_search(search_term):

    query_bow = dictionary.doc2bow(spacy_tokenizer(search_term))
    query_tfidf = df2_tfidf_model[query_bow]
    query_lsi = df2_lsi_model[query_tfidf]

    df2_index.num_best = 100000

    data_list = df2_index[query_lsi]

    data_list.sort(key=itemgetter(1), reverse=True)
    data_info = []

    for j, data in enumerate(data_list):

        data_info.append (
            {
                
                'Title': df2['title'][data[0]],
                'field_solutiondescription': df2['field_solutiondescription'][data[0]],
                'Image Description': df2['field_solutiondescriptionimage'][data[0]],
                'nid': df2['nid'][data[0]] 
            }

        )
        if j == (df2_index.num_best-1):
            break

    return pd.DataFrame(data_info, columns=['Title','field_solutiondescription','Image Description','nid'])


In [ ]:
semantic_search('machine learning classification')

,Title,field_solutiondescription,Image Description,nid
0,iris_classification,iris_classification,/sites/default/files/img_37.jpg,5829
1,iris_classification,iris_classification,/sites/default/files/img_15.jpg,6496
2,iris_classification,iris_classification,/sites/default/files/img_43.jpg,5660
3,iris_classification,iris_classification,/sites/default/files/img_36.jpg,5685
4,iris_classification,iris_classification,/sites/default/files/img_67.jpg,5697
...,...,...,...,...
3231,Supplier Operational Performance,lt p style quot margin bottom 7px text align j...,/sites/default/files/2020-04/img_59.jpg,45
3232,NLP_Imported,nlp_imported,/sites/default/files/img_91.jpg,5831
3233,NLP_Imported,nlp_imported,/sites/default/files/img_39.jpg,4190
3234,dsfdsfdfdgfd,dfdfdfdsfsd,/sites/default/files/img_79.jpg,1781


In [ ]:
#working 1 december 

To build a semantic search engine for a dataset with three columns (title, field_solutiondescription, nid), one could use a pre-trained language model to extract features from the concatenated text in the columns. The features can then be fed into a nearest neighbors algorithm to find records that are most similar to a query. This can be done using the following steps:

Import the required libraries, such as pandas, scikit-learn, and transformers.
Load the dataset into a pandas dataframe, and concatenate the text in the three columns into a single field.
Use a TfidfVectorizer to extract features from the concatenated field.
Use a NearestNeighbors algorithm to find the records that are most similar to a query.
Return the results of the nearest neighbors search in a user-friendly format, such as a pandas dataframe or a list of dictionaries.
Alternatively, one could use a deep learning model, such as BERT, to generate embeddings for the text in the dataset and use a similarity measure, such as cosine similarity, to find records that are most similar to a query.

The code above is a simple implementation of a semantic search engine. A semantic search engine attempts to understand the user's query and match it to relevant documents in a dataset.

The code first extracts the 'title' and 'field_solutiondescription' columns from the input dataframe df and concatenates them into a single column called 'data_concat'. It then removes any rows in df that contain NaN values and resets the indices.

Next, the code uses a TfidfVectorizer to extract features from the 'data_concat' column. A TfidfVectorizer is a tool used in natural language processing to convert text into numerical vectors that can be used in machine learning algorithms. It uses the tf-idf weighting scheme to assign importance to each word in the text, with words that appear frequently in the document but not across the entire dataset being given a higher weight.

The code then uses a NearestNeighbors algorithm to find the records in df that are most similar to a given query. NearestNeighbors is a type of instance-based learning algorithm that identifies the closest matches to a query based on the feature vectors generated by the TfidfVectorizer.

Finally, the code defines a function semantic_search() that takes a query as input and returns the most similar records from df based on the results of the NearestNeighbors algorithm. The function first converts the query into a numerical vector using the TfidfVectorizer, then uses the search object to find the nearest neighbors to the query in the index of vectors generated earlier. It then returns the relevant rows from df.





In [76]:
import pandas as pd

import pyarrow as pa
import datasets

import pandas as pd
df = pd.read_csv("/content/24.1_november_cleanedData (2).csv")
df = df[['title', 'field_solutiondescription','Key-Words','nid']]
df.head()

,title,field_solutiondescription,Key-Words,nid
0,$finance,sector financial servicesdata category financi...,"{'sector': 'financial servicesdata', 'category...",6533
1,House_price_prediction_final,house_price_prediction_final,['house_price_prediction_final'],6532
2,Diabetes_Patients,sector lifescience healthcaredata category hea...,"{'sector': 'lifescience healthcaredata', 'cate...",6531
3,21nov,sector business servicesdata category adaption...,"{'sector': 'business servicesdata', 'category'...",6530
4,21nov,sector business servicesdata category adaption...,"{'sector': 'business servicesdata', 'category'...",6529


In [77]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

 
# extract the text and description fields from the data frame
text = df['title']
description = df['field_solutiondescription']

# concatenate the text and description fields into a single field
df['data_concat'] = text + ' ' + description

 
# remove rows that contain NaN values and reset the indices
df = df.dropna().reset_index(drop=True)

# use a TfidfVectorizer to extract features from the concatenated field
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(df['data_concat'])

# use a NearestNeighbors algorithm to find the records that are most similar to a query
search = NearestNeighbors(n_neighbors=100)
search.fit(features)

# define a function that takes a query as input and returns the most similar records from the data frame
def semantic_search(query):
    query_features = vectorizer.transform([query])
    distances, indices = search.kneighbors(query_features)
    return df.loc[indices[0], :]

In [91]:
# define the query
query = 'machine learning classification'

# search for similar records in the data frame
results = semantic_search(query)

# save the results as a CSV file
results.to_csv('results.csv')

results.head()

,title,field_solutiondescription,Key-Words,nid,data_concat
3130,Operation Monitoring,operation monitoring solution gains real time ...,"['operation', 'monitoring', 'solution', 'gains...",248,Operation Monitoring operation monitoring solu...
2175,Learning and Development,dashboard provides insights learning developme...,"['dashboard', 'provides', 'insights', 'learnin...",2231,Learning and Development dashboard provides in...
2329,IrisExp,machine learning model file trained recognize ...,"['machine', 'learning', 'model', 'file', 'trai...",1965,IrisExp machine learning model file trained re...
2328,IrisExp,machine learning model file trained recognize ...,"['machine', 'learning', 'model', 'file', 'trai...",1966,IrisExp machine learning model file trained re...
2319,logreg,machine learning model file trained recognize ...,"['machine', 'learning', 'model', 'file', 'trai...",1995,logreg machine learning model file trained rec...


In [92]:
print(results.shape)


(100, 5)


In [93]:
from IPython.display import display

# load the CSV file into a Pandas data frame
df22 = pd.read_csv('results.csv')

# display the full data frame
display(df22)


,Unnamed: 0,title,field_solutiondescription,Key-Words,nid,data_concat
0,3130,Operation Monitoring,operation monitoring solution gains real time ...,"['operation', 'monitoring', 'solution', 'gains...",248,Operation Monitoring operation monitoring solu...
1,2175,Learning and Development,dashboard provides insights learning developme...,"['dashboard', 'provides', 'insights', 'learnin...",2231,Learning and Development dashboard provides in...
2,2329,IrisExp,machine learning model file trained recognize ...,"['machine', 'learning', 'model', 'file', 'trai...",1965,IrisExp machine learning model file trained re...
3,2328,IrisExp,machine learning model file trained recognize ...,"['machine', 'learning', 'model', 'file', 'trai...",1966,IrisExp machine learning model file trained re...
4,2319,logreg,machine learning model file trained recognize ...,"['machine', 'learning', 'model', 'file', 'trai...",1995,logreg machine learning model file trained rec...
...,...,...,...,...,...,...
95,2483,2018 Historical Hourly Weather Data for 50 U.S...,complimentary sample past historical hourly we...,"['complimentary', 'sample', 'past', 'historica...",1728,2018 Historical Hourly Weather Data for 50 U.S...
96,2385,2018 Historical Hourly Weather Data for 50 U.S...,complimentary sample past historical hourly we...,"['complimentary', 'sample', 'past', 'historica...",1861,2018 Historical Hourly Weather Data for 50 U.S...
97,2134,2018 Historical Hourly Weather Data for 50 U.S...,complimentary sample past historical hourly we...,"['complimentary', 'sample', 'past', 'historica...",2319,2018 Historical Hourly Weather Data for 50 U.S...
98,1868,2018 Historical Hourly Weather Data for 50 U.S...,complimentary sample past historical hourly we...,"['complimentary', 'sample', 'past', 'historica...",3067,2018 Historical Hourly Weather Data for 50 U.S...


In [94]:
# !pip install gradio

import gradio as gr

# define the input and output components of the app
# define the input and output components of the app
# define the input and output components of the app
inputs = gr.inputs.Textbox(label="Enter a query:")
outputs = gr.outputs.Dataframe(label="Results", type="pandas", max_rows=100, max_cols=10)


# define the app's function
def semantic_search_app(query):
    results = semantic_search(query)
    return results

# launch the app
gr.Interface(
    semantic_search_app,
    inputs=inputs,
    outputs=outputs,
    title="Semantic Search",
    description="Search for similar records in a dataset using natural language processing"
).launch()


/usr/local/lib/python3.8/dist-packages/gradio/inputs.py:26: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.8/dist-packages/gradio/outputs.py:127: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

*****using faiss cpu - 1 decemeber **bold text**

To build a semantic search engine using the faiss library, you can follow these steps:

Install the faiss library by running pip install faiss

Import the necessary modules from the faiss library, including faiss.IndexFlat, which is the main indexing class used for building a search engine.

Load the data into a Pandas dataframe, and select the columns you want to use for building the search engine. In this case, the columns are title, field_solutiondescription, Key-Words, and nid.

Convert the dataframe to a NumPy array using the .values attribute.

Create an instance of the faiss.IndexFlat class, and specify the dimension of the data (which is equal to the number of columns in the dataframe).

Add the rows of the dataframe to the index using the .add() method.

To perform a search, you can use the .search() method, which takes a query vector as input and returns a list of indices of the most similar records in the index.

You can then use these indices to retrieve the corresponding records from the dataframe and display the results.

This is just a high-level overview of the steps involved in building a semantic search engine using the faiss library. You may need to do some preprocessing of the data, such as converting text fields to numerical vectors using a vectorizer, and you may also want to experiment with different indexing algorithms and hyperparameters to improve the performance of the search engine.

In [ ]:
##25 november logic

In [ ]:
# https://www.kaggle.com/code/ajitrajput/semantic-search-engine-using-nlp/notebook

In [ ]:
import pandas as pd
df = pd.read_csv("/content/24.1_november_cleanedData (2).csv")
df1 = df[['title', 'field_solutiondescriptionimage','Key-Words','nid']]
df1.head()

,title,field_solutiondescriptionimage,Key-Words,nid
0,$finance,/sites/default/files/img_35.jpg,"{'sector': 'financial servicesdata', 'category...",6533
1,House_price_prediction_final,/sites/default/files/img_74.jpg,['house_price_prediction_final'],6532
2,Diabetes_Patients,/sites/default/files/img_80.jpg,"{'sector': 'lifescience healthcaredata', 'cate...",6531
3,21nov,/sites/default/files/img_85.jpg,"{'sector': 'business servicesdata', 'category'...",6530
4,21nov,/sites/default/files/img_58.jpg,"{'sector': 'business servicesdata', 'category'...",6529


In [95]:
def search_for_Keyword(dataframe, search_string):
    search=search_string.lower().split()
    for i in search:
        if i in list(stopwords.words('english')):
            search.remove(i)
    #print(search)
    lengths = {}
    indices_dict = {}
    for j in range(len(search)):
        indices = []
        for i in range(len(df)):
            if (search[j] in df['title'][i]) or (search[j] in df['Key-Words'][i]):
                indices.append(i)
        indices_dict[search[j]] = indices
        lengths[search[j]] = len(indices)
    
    keyword = [i for i in lengths if lengths[i]==max(lengths.values())]
    
    print("The complete items found are " + str(max(lengths.values())) + " for "+keyword[0]+". The top " +"paths are: \n")
    for i in indices_dict[keyword[0]]:  #top n values to display
        print(df["title"][i])
        print(df["field_solutiondescription"][i])
        print(df["field_solutiondescriptionimage"][i])
        print(df["nid"][i])
        print('\n\n')
        
    """    
    for i in range(len(dataframe)):
        if (keyword in df['title'][i]) or (keyword in df['Key-Words'][i]):
            indices.append(i)
    length1 = len(indices)
    #print(length1)
    #for i in range(5):
    #print(df["field_solutiondescriptionimage"][indices[i]])
    
    if length1 >= 5:
        print("The complete items found are " + str(length1) + ". The top "+"5 "+"paths are: \n")
        for i in range(5):
            print(df["field_solutiondescriptionimage"][indices[i]])
    elif (len(indices) >= 1) and (len(indices) < 5):
        print("The complete items found are " + str(length1) + ". The top "+str(length1)+" paths are: \n")
        for i in range(length1):
              print(df["field_solutiondescriptionimage"][indices[i]])
    else:
         print("There is no content against your search!!, Try again with your search!!!")"""

In [96]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [97]:
# df = pd.read_csv("CleanedDataKeyWords.csv")

# searchStr = input("Enter search word: ") #financial data #Financial data information is very crucial for the organization

searchStr = "diabetes"
df = pd.read_csv("/content/24.1_november_cleanedData (2).csv")

search_for_Keyword(df, searchStr)

The complete items found are 3 for diabetes. The top paths are: 

Pima Indian Diabetes Predictor
pima indian diabetes predictor
/sites/default/files/img_32.jpg
6348



diabetes_model
diabetes_model
/sites/default/files/img_7.jpg
5438



az-random-forest-classifier
diabetes_model
/sites/default/files/img_95.jpg
5363



